In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from kiwipiepy import Kiwi
from wordcloud import WordCloud
from collections import Counter
import re

In [57]:
# 문자열 전처리 함수 -> 숫자, 문자, (?,!,) 제외하고 공백으로 대체
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,]+"," ", w)
    w = w.strip()
    return w

In [33]:
driver = wb.Chrome()

In [34]:
url = "https://map.naver.com/p/"
driver.get(url)

In [35]:
search = driver.find_element(By.CSS_SELECTOR,'div.input_box>input') # input까지 내려가서 인자를 전달해줘야함

In [36]:
search.send_keys("봉선동 맛집" + "\n")

In [37]:
driver.switch_to.frame('searchIframe')

In [38]:
driver.find_element(By.TAG_NAME,'body')

<selenium.webdriver.remote.webelement.WebElement (session="9671727365f39677c99b001ed0f16ff7", element="f.8ED9256FEDE9CABBB76AD161A8C873B7.d.5B627C60DB525B389A31E538BA227D89.e.85")>

In [39]:
scroll = driver.find_element(By.TAG_NAME,'body')

In [40]:
scroll.send_keys(Keys.END)

In [31]:
driver.find_element(By.CSS_SELECTOR,'a.place_bluelink.YTJkH.CtW3e')

<selenium.webdriver.remote.webelement.WebElement (session="c9c74e9453776d5ff37e308932a72a3f", element="f.76B1FE1F1EAC080387EFB1A8F1C2A448.d.D1FE0D095AA0E8737395ED77EAF98459.e.9462")>

In [22]:
button.get_attribute('href')

'https://pcmap.place.naver.com/restaurant/list?query=%EB%B4%89%EC%84%A0%EB%8F%99%20%EB%A7%9B%EC%A7%91&x=126.91281594606085&y=35.14917700000079&clientX=126.912816&clientY=35.149177&bounds=126.90810598702745%3B35.144369530206276%3B126.91774048180906%3B35.153808746055276&display=70&ts=1769499289337&additionalHeight=76&locale=ko&mapUrl=https%3A%2F%2Fmap.naver.com%2Fp%2Fsearch%2F%EB%B4%89%EC%84%A0%EB%8F%99%20%EB%A7%9B%EC%A7%91#'

In [24]:
driver.get(button.get_attribute('href'))

- 여기로 이동하면 더보기 및 리뷰정보를 확인할 수 가 없음 

In [42]:
driver.find_element(By.CSS_SELECTOR,'a.place_bluelink.YTJkH.CtW3e').click()

In [ ]:
#app-root > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(5)

In [ ]:
#app-root > div > div > div.place_fixed_maintab.place_tab_shadow.place_stuck > div > div > div > div > a:nth-child(3)

In [ ]:
#app-root > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3)

- 각 음식점 별로 리뷰의 위치가 모두 다름! > 어떻게 처리해야하나?

In [45]:
driver.find_element(By.CSS_SELECTOR,'a.tpj9w._tab-menu').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.tpj9w._tab-menu"}
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff71f2888d5
	0x7ff71f288930
	0x7ff71f06165d
	0x7ff71f0b9a33
	0x7ff71f0b9d3c
	0x7ff71f10df67
	0x7ff71f10ac97
	0x7ff71f0aac29
	0x7ff71f0aba93
	0x7ff71f5a0620
	0x7ff71f59af60
	0x7ff71f5b96c6
	0x7ff71f2a5dd4
	0x7ff71f2aed7c
	0x7ff71f291ff4
	0x7ff71f2921a5
	0x7ff71f277ed2
	0x7ffbdaa7e8d7
	0x7ffbdba4c53c


- iframe을 안바꿈....

In [46]:
driver.switch_to.default_content()

In [47]:
driver.switch_to.frame('entryIframe')

In [48]:
# 리뷰 탭 클릭
driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3)').click()

In [64]:
# 펼쳐서 보기 탭 클릭
try:
    for i in range(30):
        driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
except:
    pass

In [65]:
# 리뷰 수집
preprocess_sentence_kr(driver.find_elements(By.CSS_SELECTOR,'div.pui__vn15t2>a:nth-child(1)')[0].text)

'음식이 맛있어요. 친절합니다. 모임도 자주 하고 가족끼리도 자주 갑니다.'

In [74]:
store_name = driver.find_element(By.ID,'_header').text

In [75]:
review = driver.find_elements(By.CSS_SELECTOR,'div.pui__vn15t2>a:nth-child(1)')

In [76]:
len(review)

120

In [77]:
f = open(f"./data/{store_name} 리뷰.txt","w")

for r in review:
    f.write(preprocess_sentence_kr(r.text))
f.close()



In [78]:
f = open(f"./data/{store_name} 리뷰.txt","r")
content = f.readline()
f.close()

In [82]:
kiwi = Kiwi()

In [83]:
token = kiwi.tokenize(content)

- 리스트 형태로 저장해서 tokenize가 안됨.

In [89]:
review_list = [t.form for t in token if (t.tag == "NNG") | (t.tag == "NNP") |(t.tag == "VVA") ]

In [90]:
counter = Counter(review_list)

In [92]:
counter.most_common(50)

[('오리', 61),
 ('친절', 30),
 ('고기', 22),
 ('탕', 20),
 ('죽', 19),
 ('맛', 17),
 ('음식', 15),
 ('곳', 15),
 ('직원', 13),
 ('양', 13),
 ('흑임자', 12),
 ('구이', 10),
 ('방문', 10),
 ('주문', 10),
 ('가족', 9),
 ('가성비', 8),
 ('인', 8),
 ('서비스', 7),
 ('굿', 7),
 ('점심', 7),
 ('집', 7),
 ('맛집', 7),
 ('오리탕', 6),
 ('말', 6),
 ('밥', 6),
 ('건강', 6),
 ('모임', 5),
 ('후', 5),
 ('벨', 5),
 ('솥', 5),
 ('부모', 5),
 ('사장', 5),
 ('최고', 4),
 ('검색', 4),
 ('다음', 4),
 ('특선', 4),
 ('때', 4),
 ('사람', 4),
 ('봉선', 4),
 ('식사', 4),
 ('주방', 4),
 ('테이블', 4),
 ('반찬', 4),
 ('숯', 4),
 ('계산', 3),
 ('정도', 3),
 ('응대', 3),
 ('주차장', 3),
 ('마음', 3),
 ('골', 3)]

In [93]:
driver.quit()